In [ ]:
# To predict the missing values of yob and gender I used a Linear Regression model
# I started by predicting the missing yob values then I predicted the missing gender values
# The second prediction depends on the first one when both values are missing for a user
# Maybe doing 2 independant predictions might be better in this case

In [1]:
from sklearn.linear_model import LinearRegression
import pandas as pd
from pandas import read_csv
from sklearn import preprocessing
import numpy as np

In [2]:
#load the data
path = "user_exo_2M.csv"
file = pd.read_csv(path)

In [3]:
#convert categorical data to float to be able to train the model and predict the missing values
#save a dictionnary for each column for storing the categorical data and the corresponding float
#returns the new file containing only numbers and the different dictionnaries
def convert(file):
    le = preprocessing.LabelEncoder()
    file['domain'] = le.fit_transform(file.domain)
    class1 = le.classes_
    file['firstname']= le.fit_transform(file.firstname)
    class2 = le.classes_
    file['gender'] = le.fit_transform(file.gender.astype(str))
    class3 = le.classes_
    file = file.fillna(-1)
    dictDomain = dict(zip(le.fit_transform(class1),class1))
    dictName = dict(zip(le.fit_transform(class2),class2))
    dictGender = dict(zip(le.fit_transform(class3),class3))
    return file,dictDomain,dictName,dictGender

In [4]:
#keep only the good data without any missing values 
# I think that it's better to train the model on a database that has no missing values although we'll have less data
def filter_data(File):
    File = File[File.yob != -1]
    File = File[File.gender != 2]
    return File

In [5]:
#File without categorical data as well as the different dictionnaries
NewFile,dictDomain,dictName,dictGender = convert(file)

#We start with the yob prediction
#Get only the rows with missing yob
missing_yob = NewFile.loc[NewFile['yob'] == -1]

#data without any missing info
good_data = filter_data(NewFile)

#all columns except yob from the good data file
train_yob_X = good_data.iloc[:,1:5]
#only the yob column from the good data file
train_yob_Y = good_data.iloc[:,0]
#test data: rows with missing yob  
test_yob_X = missing_yob.drop(['yob'], axis=1)


In [6]:
print("only float dataset :\n",NewFile)
print("missing yob dataset:\n",missing_yob)
print("good dataset:\n",good_data)
print("Train X :\n",train_yob_X)
print("Train Y :\n",train_yob_Y)
print("Test X :\n",test_yob_X)

only float dataset :
           yob  domain  firstname  zipcode  gender
0        1985      39      10757  94450.0       0
1        1961      36       7717  78580.0       1
2        1977      10       1393  62640.0       2
3          -1      16      12922  94100.0       1
4          -1      11      11559  78100.0       2
...       ...     ...        ...      ...     ...
1999995  1994      11        576  62400.0       0
1999996  1997      11       7851   7200.0       1
1999997  1998      11        134  21000.0       2
1999998    -1      16      13612  26240.0       1
1999999    -1      33      14772   6670.0       2

[2000000 rows x 5 columns]
missing yob dataset:
          yob  domain  firstname  zipcode  gender
3         -1      16      12922  94100.0       1
4         -1      11      11559  78100.0       2
8         -1      24      10046   5100.0       1
9         -1      39       5769  68300.0       2
13        -1      10      13172  24100.0       1
...      ...     ...        ...   

In [7]:
#model
linreg = LinearRegression()
#train the model
linreg.fit(train_yob_X,train_yob_Y)

LinearRegression()

In [8]:
#predict on missing data
#I used round to get a number that looks like a year of birth (the integer)
df_yob = np.round(pd.DataFrame(linreg.predict(test_yob_X)))

In [9]:
#add new data to the old file
NewFile.yob[NewFile['yob'] == -1] = df_yob[0].values

<ipython-input-9-8e303e153cec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NewFile.yob[NewFile['yob'] == -1] = df_yob[0].values


In [10]:
print("File with yob predictions:\n",NewFile)

File with yob predictions:
           yob  domain  firstname  zipcode  gender
0        1985      39      10757  94450.0       0
1        1961      36       7717  78580.0       1
2        1977      10       1393  62640.0       2
3        1977      16      12922  94100.0       1
4        1977      11      11559  78100.0       2
...       ...     ...        ...      ...     ...
1999995  1994      11        576  62400.0       0
1999996  1997      11       7851   7200.0       1
1999997  1998      11        134  21000.0       2
1999998  1975      16      13612  26240.0       1
1999999  1970      33      14772   6670.0       2

[2000000 rows x 5 columns]


In [11]:
#Now moving on to the gender prediction
#Get only the rows with missing gender
missing_gender = NewFile.loc[NewFile['gender'] == 2]

#all columns except gender from the good data file
train_gender_X = good_data.iloc[:,:4]
#only the gender column from the good data file
train_gender_Y = good_data.iloc[:,4]
#test data: rows with missing gender 
test_gender_X = missing_gender.drop(['gender'], axis=1)

In [12]:
#train the model
linreg.fit(train_gender_X,train_gender_Y)

LinearRegression()

In [13]:
#predict on missing data
#I used round to get a number that looks like a year of birth (the integer)
df_gender = np.round(pd.DataFrame(linreg.predict(test_gender_X)))

In [14]:
#add new data to old file
NewFile.gender[NewFile['gender'] == 2] = df_gender[0].values

<ipython-input-14-cdaf032d717c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NewFile.gender[NewFile['gender'] == 2] = df_gender[0].values


In [15]:
print("File with all the predictions:\n",NewFile)

File with all the predictions:
           yob  domain  firstname  zipcode  gender
0        1985      39      10757  94450.0       0
1        1961      36       7717  78580.0       1
2        1977      10       1393  62640.0       1
3        1977      16      12922  94100.0       1
4        1977      11      11559  78100.0       1
...       ...     ...        ...      ...     ...
1999995  1994      11        576  62400.0       0
1999996  1997      11       7851   7200.0       1
1999997  1998      11        134  21000.0       0
1999998  1975      16      13612  26240.0       1
1999999  1970      33      14772   6670.0       1

[2000000 rows x 5 columns]


In [16]:
#put back the original categorical data for the domain, firstname and gender
#Using .map is way faster than the original pd.DataFrame.replace function
NewFile['domain'] = NewFile['domain'].map(dictDomain)
NewFile['firstname'] = NewFile['firstname'].map(dictName)
NewFile['gender'] = NewFile['gender'].map(dictGender)

In [17]:
print(NewFile)

          yob        domain        firstname  zipcode gender
0        1985   @wanadoo.fr  b08253b305fb5ec  94450.0      F
1        1961       @sfr.fr  7ff135854376850  78580.0      M
2        1977      @free.fr  172522ec1028ab7  62640.0      M
3        1977  @hotmail.com  d3ca5dde60f88db  94100.0      M
4        1977    @gmail.com  bdaae16837dd576  78100.0      M
...       ...           ...              ...      ...    ...
1999995  1994    @gmail.com  09f96867a8dc816  62400.0      F
1999996  1997    @gmail.com  823fec7a2632ea7   7200.0      M
1999997  1998    @gmail.com  029f0f59eba0903  21000.0      F
1999998  1975  @hotmail.com  deb97a759ee7b8b  26240.0      M
1999999  1970    @orange.fr  f11d689dda42279   6670.0      M

[2000000 rows x 5 columns]


In [19]:
#save to .CSV file
NewFile.to_csv('user_completed.csv',index=False)